In [12]:
# Cargar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import mlflow
import mlflow.sklearn


In [3]:
# Leer datos
df = pd.read_csv("diabetes.csv")



In [4]:
# identifica las columnas que tiene valores 0
(df==0).sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [5]:
# Reemplazar ceros en columnas específicas por NaN
cols_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin']
# No se consideran algunas, por ejemplo Pregnancies (por qué si es posible que se tengan cero embarazos)

# Reemplazar ceros por NaN para poder tratarlos como datos faltantes
for col in cols_to_clean:
    df[col] = df[col].replace(0, np.nan)

# Reemplazar NaN con la moda (valor más frecuente) de cada columna
for col in cols_to_clean:
    moda = # buscar el método más común para determinar la moda de una columna
    df[col] = df[col].fillna(moda)


In [6]:
# Separar datos
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [1]:
# agregar el traking uri de mlflow

# agregar set_experiment con nombre: ClasificadorDemoDiabetes


In [17]:
# Entrenamiento y registro con MLflow
C = 1.0
max_iter = 1000
# usar los parámetros del archivo logreg_variaciones_educativas.csv
# para los parámtros C, max_iter, solver y penalty de LogisticRegresion
# se debe genear un run por cada fila del archivo, usando sus parámetros
# Analizar que usar


# agregar la línea del start_run
with mlflow.start_run():
    # El pipeline usar StandarScaler para que todos los valore numéricos estén en la misma escala
    # no cambiar dicha línea
    # LogisticRegresion deben variar sus parámetros en cada iteración.
    pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(
                C=1.0,
                max_iter=1000,
                solver="liblinear",
                penalty="l2"
            ))
        ])
    
    # Entrenar y evaluar
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)

    # recuerde que log_param y log_metric debe ir guardando los valores
    # dependiendo del run correspondiente de cada fila
    mlflow.log_param("logreg_C", C)
    mlflow.log_param("logreg_max_iter", max_iter)
    mlflow.log_param("solver", "liblinear")
    mlflow.log_param("penalty", "l2")
    
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", prec)
    
    # Guardar el pipeline completo
    mlflow.sklearn.log_model(pipeline, "modelo_pipeline")
    
    print(" Modelo registrado en MLflow")
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f}")


✅ Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667


/home/reroes/entornos/envpy310-ia-c6/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/reroes/entornos/envpy310-ia-c6/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
